# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd 
import numpy as np

In [2]:
data =pd.read_csv('tic-tac-toe.csv')
data.head(5)

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
data.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [4]:
data.columns

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')

In [5]:
for i in data.columns:
    data[i] = np.where(data[i]== 'x', 1, data[i])
    data[i] = np.where(data[i]== 'o', 0, data[i])
    data[i] = np.where(data[i]== 'b', -1, data[i])

C:\Users\const\Anaconda3\envs\ironhack\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
data.head(5)

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,0,0,0,1,0,1
2,1,1,1,1,0,0,0,0,1,1
3,1,1,1,1,0,0,0,-1,-1,1
4,1,1,1,1,0,0,-1,0,-1,1


In [7]:
data.shape

(958, 10)

In [8]:
y = data['class']
X = data.drop(['class'], axis = 1)

In [9]:
from sklearn import preprocessing

norm = X.values
min_max_scaler = preprocessing.MinMaxScaler()
norm_scaled = min_max_scaler.fit_transform(norm)
X = pd.DataFrame(norm_scaled)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [10]:
# your code here
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=65)

In [11]:
import matplotlib.pyplot as plt
import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Create Sequential model # http://faroit.com/keras-docs/0.3.3/models/
model = Sequential([
  Dense(64, activation='relu'),
  Dense(64, activation='relu'), 
  Dense(2, activation='softmax'), 
])  

Using TensorFlow backend.


In [12]:
# Compile the model
model.compile(
  optimizer='adam', 
  loss='sparse_categorical_crossentropy', # we keep the integers as targets
  metrics=['accuracy'],
)

In [13]:
# Train the model
model.fit(X_train.values, y_train.values, nb_epoch=5, batch_size=32)

C:\Users\const\Anaconda3\envs\ironhack\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/5
766/766 [==============================] - 1s 921us/step - loss: 0.6458 - accuracy: 0.6358
Epoch 2/5
766/766 [==============================] - 0s 84us/step - loss: 0.6301 - accuracy: 0.6384
Epoch 3/5
766/766 [==============================] - 0s 87us/step - loss: 0.6157 - accuracy: 0.6540
Epoch 4/5
766/766 [==============================] - 0s 84us/step - loss: 0.6023 - accuracy: 0.6841
Epoch 5/5
766/766 [==============================] - 0s 98us/step - loss: 0.5924 - accuracy: 0.6802


In [14]:
# Evaluate the model
model.evaluate(X_test, y_test)

192/192 [==============================] - 0s 624us/step


[0.5397879282633463, 0.7708333134651184]

In [15]:
print(model.metrics_names)

['loss', 'accuracy']


In [16]:
# Save the model to disk.
model.save_weights('tic-tac-toe.model')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [17]:
# your code here
model.load_weights('tic-tac-toe.model')

In [18]:
# Prediction
predictions = model.predict(X_test.values[:5])
# Print our model's predictions.
print(np.argmax(predictions, axis=1))

[1 1 1 1 1]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [19]:
# your code here

# Create Sequential model
model = Sequential([
  Dense(64, activation='relu'),
  Dense(64, activation='relu'), 
  Dense(64, activation='relu'), 
  Dense(2, activation='softmax'), 
]) 

In [20]:
# Compile the model
from keras.optimizers import SGD

In [21]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(
  optimizer='sgd', 
  loss='sparse_categorical_crossentropy', # we keep the integers as targets
  metrics=['accuracy'],
)

In [22]:
# Train the model
model.fit(X_train.values, y_train.values, nb_epoch=9, batch_size=32)

C:\Users\const\Anaconda3\envs\ironhack\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/9
766/766 [==============================] - 0s 609us/step - loss: 0.6871 - accuracy: 0.5418
Epoch 2/9
766/766 [==============================] - 0s 62us/step - loss: 0.6610 - accuracy: 0.6345
Epoch 3/9
766/766 [==============================] - 0s 83us/step - loss: 0.6536 - accuracy: 0.6358
Epoch 4/9
766/766 [==============================] - 0s 67us/step - loss: 0.6506 - accuracy: 0.6358
Epoch 5/9
766/766 [==============================] - 0s 74us/step - loss: 0.6481 - accuracy: 0.6358
Epoch 6/9
766/766 [==============================] - 0s 73us/step - loss: 0.6463 - accuracy: 0.6358
Epoch 7/9
766/766 [==============================] - 0s 133us/step - loss: 0.6446 - accuracy: 0.6358
Epoch 8/9
766/766 [==============================] - 0s 141us/step - loss: 0.6429 - accuracy: 0.6358
Epoch 9/9
766/766 [==============================] - 0s 135us/step - loss: 0.6417 - accuracy: 0.6358


In [23]:
# Evaluate the model
model.evaluate(X_test, y_test)

192/192 [==============================] - 0s 670us/step


[0.5938595434029897, 0.7239583134651184]

In [24]:
print(model.metrics_names)

['loss', 'accuracy']


**Which approach(es) did you find helpful to improve your model performance?**